In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
#Importing the required libraries

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
from pyspark.sql.functions import datediff,date_format,to_date,to_timestamp
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder
from pyspark.ml import Pipeline

#Table Creation
df = spark.read.csv('/FileStore/tables/heartstroke.csv',inferSchema=True,header=True)
df.createOrReplaceTempView("heartstroke")

#Displaying the data
display(df)

gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
Male,58,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
Female,70,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
Female,52,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
Female,75,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
Female,32,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
Female,74,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0
Male,79,0,1,Yes,Private,Urban,57.08,22.0,formerly smoked,0
Female,37,0,0,Yes,Private,Rural,162.96,39.4,never smoked,0
Female,37,0,0,Yes,Private,Rural,73.5,26.1,formerly smoked,0
Female,40,0,0,Yes,Private,Rural,95.04,42.4,never smoked,0


In [0]:
#Feature Engineering

from pyspark.sql.functions import *
df=df.withColumn("gender", when(col("gender") == 'Male',1)
      .otherwise(0))

In [0]:
df=df.withColumn("ever_married", when(col("ever_married") == 'Yes',1)
      .otherwise(0))

In [0]:
df=df.withColumn("Residence_type", when(col("Residence_type") == 'Urban',1)
      .otherwise(0))

In [0]:
#Creating a new feature

df=df.withColumn("bmi_cat", when((col("bmi") <= 18.5), "Underweight")
                           .when(((col("bmi") <= 25) & (col("bmi") > 18.5)), "Healthy")
                           .when(((col("bmi") <= 30) & (col("bmi") > 25)), "Overweight")
                           .when((col("bmi") > 30), "Obese")
                           .otherwise(0))

In [0]:
#Correlation matrix

from pyspark.ml.stat import Correlation
Variables_corr= ['gender', 'age',
'hypertension','heart_disease','ever_married','Residence_type','avg_glucose_level','stroke']
assembler2 = VectorAssembler(inputCols=Variables_corr,outputCol="features")
output = assembler2.transform(df)
r1 = Correlation.corr(output, "features")
cor_np = r1.collect()[0][r1.columns[0]].toArray()
cor_np

Out[29]: array([[ 1.00000000e+00,  4.19614414e-02,  3.79345990e-02,
         9.79962366e-02,  2.60651098e-02,  4.96382757e-03,
         5.40051863e-02,  1.38904202e-02],
       [ 4.19614414e-02,  1.00000000e+00,  2.59527834e-01,
         2.51818656e-01,  5.46996109e-01,  4.04437054e-03,
         2.30682361e-01,  1.59837655e-01],
       [ 3.79345990e-02,  2.59527834e-01,  1.00000000e+00,
         1.14956673e-01,  1.33258377e-01, -4.42745033e-03,
         1.54701702e-01,  7.33097919e-02],
       [ 9.79962366e-02,  2.51818656e-01,  1.14956673e-01,
         1.00000000e+00,  9.82293320e-02, -5.82829159e-04,
         1.39448985e-01,  1.07007033e-01],
       [ 2.60651098e-02,  5.46996109e-01,  1.33258377e-01,
         9.82293320e-02,  1.00000000e+00,  4.99020437e-03,
         1.20160883e-01,  5.16657588e-02],
       [ 4.96382757e-03,  4.04437054e-03, -4.42745033e-03,
        -5.82829159e-04,  4.99020437e-03,  1.00000000e+00,
        -1.36061511e-03,  2.06375877e-03],
       [ 5.40051863e-02, 

In [0]:
#Displaying the data after preprocessing

display(df)

gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_cat
1,58,1,0,1,Private,1,87.96,39.2,never smoked,0,Obese
0,70,0,0,1,Private,0,69.04,35.9,formerly smoked,0,Obese
0,52,0,0,1,Private,1,77.59,17.7,formerly smoked,0,Underweight
0,75,0,1,1,Self-employed,0,243.53,27.0,never smoked,0,Overweight
0,32,0,0,1,Private,0,77.67,32.3,smokes,0,Obese
0,74,1,0,1,Self-employed,1,205.84,54.6,never smoked,0,Obese
1,79,0,1,1,Private,1,57.08,22.0,formerly smoked,0,Healthy
0,37,0,0,1,Private,0,162.96,39.4,never smoked,0,Obese
0,37,0,0,1,Private,0,73.5,26.1,formerly smoked,0,Overweight
0,40,0,0,1,Private,0,95.04,42.4,never smoked,0,Obese


In [0]:
df.schema

Out[31]: StructType(List(StructField(gender,IntegerType,false),StructField(age,IntegerType,true),StructField(hypertension,IntegerType,true),StructField(heart_disease,IntegerType,true),StructField(ever_married,IntegerType,false),StructField(work_type,StringType,true),StructField(Residence_type,IntegerType,false),StructField(avg_glucose_level,DoubleType,true),StructField(bmi,DoubleType,true),StructField(smoking_status,StringType,true),StructField(stroke,IntegerType,true),StructField(bmi_cat,StringType,false)))

In [0]:
# Removing the NA values

df = df.dropna()

In [0]:
#Splitting the data into train and test data

train_data,test_data=df.randomSplit([0.8,0.2],seed=1234)

In [0]:
x=train_data.drop('stroke')
y=train_data['stroke']

train_0=train_data[train_data['stroke']==0];
train_1=train_data[train_data['stroke']==1];
train_0.count(),train_1.count()
df_class1=train_1.sample(withReplacement=True, fraction=train_0.count()/train_1.count(),seed=1234)
train_data=train_0.union(df_class1)

In [0]:
#String Indexer

work_type_Indexer = StringIndexer(inputCol='work_type',outputCol='work_type_index',handleInvalid='keep')
smoking_status_Indexer = StringIndexer(inputCol='smoking_status',outputCol='smoking_status_index',handleInvalid='keep')
bmi_cat_Indexer = StringIndexer(inputCol='bmi_cat',outputCol='bmi_cat_index',handleInvalid='keep')

In [0]:
# Performing onehotencoding on the above features

data_encoder = OneHotEncoder(inputCols=['work_type_index','smoking_status_index','bmi_cat_index'],
                                      outputCols=['work_type_vec','smoking_status_vec','bmi_cat_ved'],
                                      handleInvalid='keep')

In [0]:
assembler = VectorAssembler(inputCols=['gender','age','hypertension','heart_disease','ever_married','Residence_type','avg_glucose_level'],
                            outputCol="features")

In [0]:
#Model Creation

from pyspark.ml.classification import RandomForestClassifier


In [0]:
for i in range(4,30):
  rf_model = RandomForestClassifier(labelCol='stroke',maxDepth=i)
  pipe = Pipeline(stages=[work_type_Indexer,smoking_status_Indexer,bmi_cat_Indexer,data_encoder,assembler,rf_model])
  fit_model=pipe.fit(train_data)
  results = fit_model.transform(test_data)
  from pyspark.ml.evaluation import MulticlassClassificationEvaluator
  ACC_evaluator = MulticlassClassificationEvaluator(
      labelCol="stroke", predictionCol="prediction", metricName="accuracy")
  accuracy = ACC_evaluator.evaluate(results)
  print("The accuracy of the model is {}".format(accuracy))

The accuracy of the model is 0.7429705019837847
The accuracy of the model is 0.7462480593410384
The accuracy of the model is 0.7633258582025185
The accuracy of the model is 0.7802311540451958
The accuracy of the model is 0.7948939106434363
The accuracy of the model is 0.811281697429705
The accuracy of the model is 0.8356046230809039
The accuracy of the model is 0.8573400034500603
The accuracy of the model is 0.8757978264619631
The accuracy of the model is 0.8818354321200621
The accuracy of the model is 0.8946006555114715
The accuracy of the model is 0.8987407279627393
The accuracy of the model is 0.900120752113162
The accuracy of the model is 0.9013282732447818
The accuracy of the model is 0.9035708124892186
The accuracy of the model is 0.9056408487148525
The accuracy of the model is 0.907193375884078
The accuracy of the model is 0.9070208728652751
The accuracy of the model is 0.907193375884078
The accuracy of the model is 0.907193375884078
The accuracy of the model is 0.90719337588407

In [0]:
results.select(['stroke','prediction']).show()

+------+----------+
|stroke|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
+------+----------+
only showing top 20 rows



In [0]:
#Accuracy

